# Gulf Deep Sea Coral Richness

This indicator was used in the Southeast Blueprint 2023.
Code written by Rua Mordecai

In [22]:
import os
import arcpy

In [23]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\ContinentalSoutheastBlueprint2023_FinalIndicators\ContinentalSoutheastBlueprint2023_FinalIndicators\SpatialData"

In [24]:
# define final output name
IndicatorFileName = "GulfDeepSeaCoralRichness.tif"

In [25]:
# define raster used for cell size, extent, and snapping
MarineRaster= r"D:\Blueprint\2023\extent\Marine_Extent_v1.tif"

In [26]:
# define inputs
coralData = r"D:\Blueprint\2023\Indicators\GulfDeepSeaCoral\Classified_Predicted_DSC_Genus_Richness_GulfOfMexico\Classified_Predicted_Genus_Richness\Classified_Predicted_DSC_Genus_Richness.shp"

In [27]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

### Start analysis

In [28]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

In [39]:
# Create dictionary to convert text descriptions into indicator values
# Then make that dictionary usable in CalculateField but putting it in a codeblock within a function
reClassCode = """
binConvert = {"0 – 0.5":1, "0.5 – 1":1, "1 – 2":2, "2 – 3":2, "3 – 4":3, "4 – 5":3,
          "5 – 6":4,"6 – 7":4,"7 – 8":5,">8":5}
def reClass(x):
    return binConvert[x]"""

In [40]:
# Make a copy of input data so I don't mess with the original
arcpy.CopyFeatures_management(coralData, "coral.shp")

<Result 'D:\\Blueprint\\2023\\Indicators\\GulfDeepSeaCoral\\working\\coral.shp'>

In [41]:
# add a field to convert to raster and calculate the raster value based on dictionary above
arcpy.management.CalculateField("coral.shp", "raster", "reClass(!GENUSRICH!)", "PYTHON3", reClassCode, "SHORT")

<Result 'D:\\Blueprint\\2023\\Indicators\\GulfDeepSeaCoral\\working\\coral.shp'>

In [20]:
# convert vector to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    arcpy.conversion.FeatureToRaster(in_features="coral.shp", field="raster", out_raster="coralClasses.tif", cell_size=MarineRaster)

In [18]:
# export nonclipped version of indicator
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    arcpy.management.CopyRaster("coralClasses.tif", IndicatorFileName, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [43]:
# clip raster to Marine extent
# out_raster = arcpy.sa.ExtractByMask("coralClasses.tif", MarineRaster); out_raster.save(IndicatorFileName)

### Finalize indicator



In [29]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 5:
        return '5 = Highest predicted average genus richness (>7)'
    elif value == 4:
        return '4 = High predicted average genus richness (>5-7)'
    elif value == 3:
        return '3 = Medium predicted average genus richness (>3-5)'
    elif value == 2:
        return '2 = Low predicted average genus richness (>1-3)'
    elif value == 1:
        return '1 = Lowest predicted average genus richness (0-1)'      
"""

In [30]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\ContinentalSoutheastBlueprint2023_FinalIndicators\\ContinentalSoutheastBlueprint2023_FinalIndicators\\SpatialData\\GulfDeepSeaCoralRichness.tif'>

In [31]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 103
	if Value == 4:
		return 145
	if Value == 3:
		return 186
	if Value == 2:
		return 214
	if Value == 1:
		return 235
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 34
	if Value == 4:
		return 56
	if Value == 3:
		return 82
	if Value == 2:
		return 118
	if Value == 1:
		return 158
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 63
	if Value == 4:
		return 87
	if Value == 3:
		return 111
	if Value == 2:
		return 137
	if Value == 1:
		return 167
	else:
		return 255
"""

In [32]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\ContinentalSoutheastBlueprint2023_FinalIndicators\\ContinentalSoutheastBlueprint2023_FinalIndicators\\SpatialData\\GulfDeepSeaCoralRichness.tif'>